# Project: Pandas dataframe data consumption omptimization (from Dataquest data engineer course)
## Introduction
In this project, assumme, there is limitation by memory which can be used to conduct analysis on Dataset. Lets say, only 20 MB of RAM, that can be used to store dataframe.

Loans dataset is given. The goal is to choose the correct datatype for each column to reduce overall memory consumption. It is not essential, what actual data is in the dataset. Only its data types matters.

So, this is stated:  
* **Problem**: 20 MB memory limitation  
* **Goal**: Reduce dataframe memory consuption as much as possible  
* **Approach**:
    * Clean corrupted values
    * Format columns
    * Determine appropriate data types to convert. Data types used: *Category, float, int, datetime, bool*.
    * Process in chunks to fit in memory limitation 

Lets start by importing libraries, declaring a couple variables and overviewing dataset itself.

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 99
LOANS_CSV_PATH = "loans_2007.csv"

In [3]:
loans_5 = pd.read_csv(LOANS_CSV_PATH, nrows=5)
loans_5

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## 1. Dataframe values observation
### Since there is 20 MB memory limit, only 5 MB will be used to store dataframe in order to leave some memory for the rest.  

In [4]:
loans_1000 = pd.read_csv(LOANS_CSV_PATH, nrows=1000)
mem_usage_mb = loans_1000.memory_usage(deep=True).sum() / 2**20
print(mem_usage_mb)

1.5273704528808594


### One thousand rows consumes 1.5 MB of memory. Hence, 3000 rows chunks will be appropriate size for processing.

In [5]:
loans_3000 = pd.read_csv(LOANS_CSV_PATH, nrows=3000)
mem_usage_mb = loans_3000.memory_usage(deep=True).sum() / 2**20
print(mem_usage_mb)

4.5803985595703125


### After establishing an appropiate size of chunks, determining default amount of *numeric* and *object* columns.

In [6]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)

numeric = []
strings = []

for chunk in loans_chunks:
    numeric_dtypes = chunk.select_dtypes(include=np.number).shape[1]
    numeric.append(numeric_dtypes)

    string_dtypes = chunk.select_dtypes(include=["object"])
    strings.append(string_dtypes.shape[1])

chunks_amount = len(numeric)
print(strings)
print(numeric)

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]


### There is an unexpected difference in columns amount for two last chunks. Exploring, which column causes this difference and why does it happen?

In [7]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)

expected_strings_columns = []
actual_strings_columns = []
corrupted_chunks = []

for chunk in loans_chunks:
    string_columns = chunk.select_dtypes(include="object").columns
    actual_strings_columns = string_columns.tolist()
    if len(expected_strings_columns) == 0:
        expected_strings_columns = string_columns.tolist()

    if expected_strings_columns != actual_strings_columns:
        corrupted_chunks.append(chunk)
        for column in actual_strings_columns:
            if column not in expected_strings_columns:
                print(column)

id
id


### Seems, that "id" column has incorrect values, that makes chunk determine it's data type as *object*. Trying to figure out, which particular values causes this behavior.

In [8]:
corrupted_values = []

for chunk in corrupted_chunks:
    for value in chunk["id"].values:
        if not value.isdigit():
            corrupted_values.append(value)
            print(value)

Loans that do not meet the credit policy
Total amount funded in policy code 1: 471701350
Total amount funded in policy code 2: 0


### Defining a function, which will be used everytime while iterating over chunks to clean them from corrupted values.

In [9]:
def fix_chunk(chunk):
    chunk = chunk[
        ~chunk["id"].isin(corrupted_values)
    ].copy()  # Filtering out non-int values
    chunk["id"] = chunk["id"].astype(
        np.float64
    )
    return chunk

## 2. Determining appropriate data types
### Now, it is possible to start determining columns, which memory consuption can be optimized. Lets start with *object* columns and figure out which can be converted to *category* data type.

In [10]:
import numpy as np

loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)
unique_values_by_chunks = {}
n_rows = 0

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)
    columns = chunk.select_dtypes(include="object").columns
    n_rows += len(chunk.index)

    for col in columns:
        col_vc = chunk[col].value_counts()
        if col in unique_values_by_chunks:
            unique_values_by_chunks[col].append(col_vc)
        else:
            unique_values_by_chunks[col] = [col_vc]

unique_values_vc = {}
str_columns_meta = {}

for key in unique_values_by_chunks:
    column_df = pd.concat(unique_values_by_chunks[key])
    unique_values_vc[key] = column_df.groupby(column_df.index).sum()

    values = unique_values_vc[key].index
    column_uniques = len(values)
    str_columns_meta[key] = {
        "n_uniques": column_uniques,
        "percent_uniques": column_uniques / n_rows,
        "suitable_for_cat": column_uniques / n_rows < 0.5,
        "values": values.tolist(),
    }

### Finding *numeric* columns, that does not contain NaN values, so they can be converted to *int* data type.

In [11]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)

n_numeric_nan_by_chunks = []

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)
    numeric_columns = chunk.select_dtypes(include="float")
    n_numeric_nan_by_chunks.append(numeric_columns.isnull().sum())

n_numeric_nan_list = pd.concat(n_numeric_nan_by_chunks)
n_numeric_nan = n_numeric_nan_list.groupby(n_numeric_nan_list.index).sum()
no_nan_float_columns = n_numeric_nan[n_numeric_nan == 0].index.tolist()

print(no_nan_float_columns)

['collection_recovery_fee', 'dti', 'funded_amnt', 'funded_amnt_inv', 'id', 'installment', 'last_pymnt_amnt', 'loan_amnt', 'member_id', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp']


In [12]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)
to_int_candidates = no_nan_float_columns.copy()

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)
    chunk_reduced = chunk[to_int_candidates]  # Leave columns, which still can be ints
    for col in chunk_reduced.columns:
        col_s = chunk_reduced[col]
        if not np.array_equal(col_s, col_s.astype(int)):
            to_int_candidates.remove(col)

print(to_int_candidates)

['funded_amnt', 'id', 'loan_amnt', 'member_id', 'policy_code', 'revol_bal']


### Lets now recall *object* data type columns and watch, are there any columns, which somehow can be converted to *numeric* data type.

In [37]:
import json
df = pd.read_csv(LOANS_CSV_PATH, nrows=1000)

df[list(str_columns_meta.keys())]

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type
0,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,Jun-2016,INDIVIDUAL
1,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,Sep-2013,INDIVIDUAL
2,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,Jun-2016,INDIVIDUAL
3,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,Feb-1996,21%,f,Jan-2015,Apr-2016,INDIVIDUAL
4,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,Jan-1996,53.9%,f,Jun-2016,Jun-2016,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,36 months,15.27%,C,C4,Lime Energy,3 years,OWN,Not Verified,Dec-2011,Fully Paid,n,home_improvement,top up,088xx,NJ,Jul-2000,76.8%,f,Jan-2015,Jul-2015,INDIVIDUAL
996,36 months,9.91%,B,B1,Real Mex Foods,2 years,RENT,Not Verified,Dec-2011,Fully Paid,n,credit_card,Credit Card Loan,916xx,CA,Jul-2004,83%,f,Sep-2012,Apr-2014,INDIVIDUAL
997,36 months,9.91%,B,B1,JP Morgan Chase,10+ years,OWN,Verified,Dec-2011,Fully Paid,n,credit_card,Relief Refi,322xx,FL,Mar-1995,79.4%,f,Dec-2012,Jun-2016,INDIVIDUAL
998,60 months,20.30%,E,E5,Regional Transportation District,10+ years,MORTGAGE,Verified,Dec-2011,Fully Paid,n,debt_consolidation,CC Consolidation,802xx,CO,Jan-2001,66%,f,Jan-2016,Feb-2016,INDIVIDUAL


### Exploring columns, that theoretically can be converted to other data type but *object*, list bellow was created.  
**Columns to convert**:
1. Numeric:
    * *term*: leave only months amount; convert to int
    * *int_rate*: remove % sign; convert to float
    * *revol_util*: remove % sign; convert to float
2. Datetime
    * *issue_d*: convert to date
    * *earliest_cr_line*: convert to date
    * *last_pymnt_d*: convert to date
    * *last_credit_pull_d*: convert to date
3. Bool:
    * *pymnt_plan*: map "y" to True, "n" to False; convert to bool
4. Categorical
    * *others*: convert to categorical if suitable

## 3. Starting optimization
### As long as "term" column theoreticaly can be converted to *int* data type, it must be checked for NaN values abscense.

In [14]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)
nan_amnt = 0

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)
    term_column = chunk['term']
    nan_amnt += term_column.isnull().sum()

print(nan_amnt)

0


### Now, let's summarize columns and its data types in python dictionary. 

In [15]:
clean_to_float = ["int_rate", "revol_util"]  # remove % sign; rename column; convert to float
to_date = [
    "issue_d",
    "earliest_cr_line",
    "last_pymnt_d",
    "last_credit_pull_d",
]  # parse dates
to_bool = ["pymnt_plan"]  # has only "y" and "n" values
clean_to_int = ["term"]  # remove "months" word; rename column; convert to int.

str_columns_set = set(
    [key for key in str_columns_meta if str_columns_meta[key]["suitable_for_cat"]]
)
to_cat = list(
    str_columns_set.difference(
        clean_to_float + to_date + to_bool + clean_to_int
    )
)

casts_d = {
    "clean": {"float": clean_to_float, "int": clean_to_int},
    "date": to_date,
    "bool": to_bool,
    "int": to_int_candidates,
    "categorical": to_cat,
}

### Before optimization process, calculate dataframe total memory consumption by default. 

In [16]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)

total_mem_usage_mb = 0

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)
    total_mem_usage_mb += chunk.memory_usage(deep=True).sum() / 2**20

print(total_mem_usage_mb)

65.08100986480713


### Format columns where needed and convert them to appropriate data type.

In [17]:
loans_chunks = pd.read_csv(LOANS_CSV_PATH, chunksize=3000)

memory_usage_optimized = 0

for chunk in loans_chunks:
    chunk = fix_chunk(chunk)

    cols_to_convert = casts_d["clean"]["float"]
    for col in cols_to_convert:
        col_values = chunk[col].dropna()
        chunk[col] = col_values.apply(lambda val: val.strip("% ")).astype("float")
        chunk.rename(columns={col: col + "%"}, inplace=True)

    cols_to_convert = casts_d["clean"]["int"]
    for col in cols_to_convert:
        col_values = chunk[col]
        chunk[col] = col_values.apply(lambda val: val.strip("months ")).astype("int")
        chunk.rename(columns={col: col + "_months"}, inplace=True)

    cols_to_convert = casts_d["int"]
    for col in cols_to_convert:
        chunk[col] = chunk[col].astype("int")

    cols_to_convert = casts_d["date"]
    for col in cols_to_convert:
        col_values = chunk[col]
        col_values = pd.to_datetime(col_values, format="%b-%Y")
        chunk[col] = col_values

    cols_to_convert = casts_d["bool"]
    for col in cols_to_convert:
        col_values = chunk[col]
        col_values = col_values.map({"y": True, "n": False})
        chunk[col] = col_values

    cols_to_convert = casts_d["categorical"]
    for col in cols_to_convert:
        col_values = chunk[col]
        chunk[col] = col_values.astype("category")

    int_cols = chunk.select_dtypes(include="int").columns
    float_cols = chunk.select_dtypes(include="float").columns

    chunk[int_cols] = chunk[int_cols].apply(pd.to_numeric, downcast="integer")
    chunk[float_cols] = chunk[float_cols].apply(pd.to_numeric, downcast="float")

    memory_usage_optimized += chunk.memory_usage(deep=True).sum()

print(memory_usage_optimized / 2**20)

14.739266395568848


## Summary:
Before optimization, dataframe consumed 65 MB of memory, after optimization process it consumes 14.7 MB. Thus, dataframe memory consuption was decreased by **366%!** It is possible, since appropriate data types were chosen.

Obviously, nowadays saving 50 MB memory is not a huge advantage, but imagine what if it was gigabytes instead?

This approach can be very useful in systems, where memory usage is essential. Futhermore, the analysys process becomes much faster, since several columns became *Numeric* datatype instead of *Object*, which works much faster with NumPy library.